In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt

import operator
from functools import reduce
from functools import partial
from timeit import default_timer

torch.manual_seed(0)
np.random.seed(0)


#  1d fourier layer

class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  #Number of Fourier modes to multiply, at most floor(N/2) + 1

        self.scale = (1 / (in_channels*out_channels))
        self.weights1 = nn.Parameter(self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul1d(self, input, weights):
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x, out_ft0 = None):
        batchsize = x.shape[0]
        #Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft(x)

        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1)//2 + 1,  device=x.device, dtype=torch.cfloat)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :, :self.modes1], self.weights1)
        if out_ft0 is not None:
          out_ft0 += out_ft
        else:
          out_ft0 = out_ft

        #Return to physical space
        x = torch.fft.irfft(out_ft0, n=x.size(-1))
        return x, out_ft

class FNO1d(nn.Module):
    def __init__(self, modes, width):
        super(FNO1d, self).__init__()

        """
        The overall network. It contains 4 layers of the Fourier layer.
        1. Lift the input to the desire channel dimension by self.fc0 .
        2. 4 layers of the integral operators u' = (W + K)(u).
            W defined by self.w; K defined by self.conv .
        3. Project from the channel space to the output space by self.fc1 and self.fc2 .
        
        input: the solution of the initial condition and location (a(x), x)
        input shape: (batchsize, x=s, c=2)
        output: the solution of a later timestep
        output shape: (batchsize, x=s, c=1)
        """

        self.modes1 = modes
        self.width = width
        self.fc0 = nn.Linear(1, self.width) # input channel is 2: (a(x), x)

        self.conv0 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv1 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv2 = SpectralConv1d(self.width, self.width, self.modes1)
        self.conv3 = SpectralConv1d(self.width, self.width, self.modes1)
        self.w0 = nn.Conv1d(self.width, self.width, 1)
        self.w1 = nn.Conv1d(self.width, self.width, 1)
        self.w2 = nn.Conv1d(self.width, self.width, 1)
        self.w3 = nn.Conv1d(self.width, self.width, 1)


        self.fc1 = nn.Linear(self.width, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x,  out_ft0 = None, out_ft1 = None, out_ft2 = None, out_ft3 = None):

        x = self.fc0(x)
        x = x.permute(0, 2, 1)

        if out_ft0 is None:

          x1, out_ft0 = self.conv0(x)
          x2 = self.w0(x)
          x = x1 + x2
          x = F.relu(x)
        else:
          x1, out_ft0 = self.conv0(x, out_ft0)
          x2 = self.w0(x)
          x = x1 + x2
          x = F.relu(x)
        
       if out_ft1 is None:
          x1, out_ft1 = self.conv1(x)
          x2 = self.w1(x)
          x = x1 + x2
          x = F.relu(x)
       else:
          x1, out_ft1 = self.conv1(x, out_ft1)
          x2 = self.w1(x)
          x = x1 + x2
          x = F.relu(x)
        
        if out_ft2 is None:
          x1, out_ft2 = self.conv2(x)
          x2 = self.w2(x)
          x = x1 + x2
          x = F.relu(x)
        else:
          x1, out_ft2 = self.conv2(x, out_ft2)
          x2 = self.w2(x)
          x = x1 + x2
          x = F.relu(x)
        
        if out_ft3 is None:
          x1, out_ft3 = self.conv3(x)
          x2 = self.w3(x)
          x = x1 + x2
        else:
          x1, out_ft3 = self.conv3(x, out_ft3)
          x2 = self.w3(x)
          x = x1 + x2
          


        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x, out_ft0, out_ft1, out_ft2, out_ft3

In [ ]:
# Coupled Data
ntrain = 1000
ntest = 200

sub = 2**0 #subsampling rate
h = 2**8 // sub #total grid size divided by the subsampling rate
s = h
batch_size = 20

rw_u = loadmat('/content/drive/MyDrive/gray_scott_results/Coupled_PDE_data/kernel1Drho_t0_1.mat')
x_data = rw_u['rho_t0'].astype(np.float32)
y_data = rw_u['rho_t02'].astype(np.float32)
print(x_data.shape)

x_train_u = x_data[:ntrain,::sub]
y_train_u = y_data[:ntrain,::sub]
x_test_u = x_data[-ntest:,::sub]
y_test_u = y_data[-ntest:,::sub]

x_train_u = torch.from_numpy(x_train_u)
x_test_u = torch.from_numpy(x_test_u)
y_train_u = torch.from_numpy(y_train_u)
y_test_u = torch.from_numpy(y_test_u)

x_train_u = x_train_u.unsqueeze(-1)
x_test_u = x_test_u.unsqueeze(-1)

rw_v = loadmat('/content/drive/MyDrive/gray_scott_results/Coupled_PDE_data/kernel1Dphi_t0_1.mat')
x_data = rw_v['phi_t0'].astype(np.float32)
y_data = rw_v['phi_t02'].astype(np.float32)

x_train_v = x_data[:ntrain,::sub]
y_train_v = y_data[:ntrain,::sub]
x_test_v = x_data[-ntest:,::sub]
y_test_v = y_data[-ntest:,::sub]

x_train_v = torch.from_numpy(x_train_v)
x_test_v = torch.from_numpy(x_test_v)
y_train_v = torch.from_numpy(y_train_v)
y_test_v = torch.from_numpy(y_test_v)
print(y_test_u.shape)

x_train_v = x_train_v.unsqueeze(-1)
x_test_v = x_test_v.unsqueeze(-1)

x_train = torch.cat([x_train_u.reshape(ntrain,s,1), x_train_v.reshape(ntrain,s,1)], dim=2)
x_test = torch.cat([x_test_u.reshape(ntest,s,1), x_test_v.reshape(ntest,s,1)], dim=2)

y_train = torch.cat([y_train_u.reshape(ntrain,s,1), y_train_v.reshape(ntrain,s,1)], dim=2)
y_test = torch.cat([y_test_u.reshape(ntest,s,1), y_test_v.reshape(ntest,s,1)], dim=2)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

In [ ]:
# model
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

modes = 16
width = 64

model1 = FNO1d(modes, width).cuda()
model2 = FNO1d(modes, width).cuda()
print(count_params(model1))

287361


In [ ]:
import random

optimizer1 = torch.optim.Adam(model1.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=step_size, gamma=gamma)

optimizer2 = torch.optim.Adam(model2.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler2 = torch.optim.lr_scheduler.StepLR(optimizer2, step_size=step_size, gamma=gamma)

In [ ]:
# Train and test
error_u = []
error_v = []
mae_error_u = []
mae_error_v = []


# Train and test
error_u = []
error_v = []

myloss = LpLoss(size_average=False)
len_loader = ntrain/batch_size

error = []

for ep in range(epochs):
    model1.train()
    model2.train()
    t1 = default_timer()
    train_l2 = 0

    train_l2_u = 0
    train_l2_v = 0

    flag_1 = 0
    flag_2 = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()

        random_num = random.random()
        if random_num < 0.5:
            with torch.no_grad(): 
              out1, out_ft0, out_ft1, out_ft2, out_ft3 = model1(x[:,:,0].unsqueeze(-1))
            optimizer2.zero_grad()

            v_out_end, out_ft0, out_ft1, out_ft2, out_ft3 = model2(x[:,:,1].unsqueeze(-1), out_ft0= None, out_ft1=None, out_ft2=None, out_ft3=out_ft3)

            l2_v = myloss(v_out_end.view(batch_size, -1), y[:,:,1].view(batch_size, -1))
            l2_v.backward() # use the l2 relative loss

            optimizer2.step()
            train_l2_v += l2_v.item()

            with torch.no_grad():
               out2, out_ft0, out_ft1, out_ft2, out_ft3 = model2(x[:,:,1].unsqueeze(-1))

            optimizer1.zero_grad()

            u_out_end, out_ft0, out_ft1, out_ft2, out_ft3 = model1(x[:,:,0].unsqueeze(-1), out_ft0= None, out_ft1=None, out_ft2=None, out_ft3=out_ft3)
            l2_u = myloss(u_out_end.view(batch_size, -1), y[:,:,0].view(batch_size, -1))
            l2_u.backward() # use the l2 relative loss

            optimizer1.step()
            train_l2_u += l2_u.item()
        else:

            with torch.no_grad(): 
              out2, out_ft0, out_ft1, out_ft2, out_ft3 = model2(x[:,:,1].unsqueeze(-1))
            optimizer1.zero_grad()

            u_out_end, out_ft0, out_ft1, out_ft2, out_ft3 = model1(x[:,:,0].unsqueeze(-1), out_ft0= None, out_ft1=None, out_ft2=None, out_ft3=out_ft3)

            l2_u = myloss(u_out_end.view(batch_size, -1), y[:,:,0].view(batch_size, -1))
            l2_u.backward() # use the l2 relative loss

            optimizer1.step()
            train_l2_u += l2_u.item()

            with torch.no_grad(): 
               out1, out_ft0, out_ft1, out_ft2, out_ft3 = model1(x[:,:,0].unsqueeze(-1))
            optimizer2.zero_grad()

            v_out_end, out_ft0, out_ft1, out_ft2, out_ft3 = model2(x[:,:,1].unsqueeze(-1), out_ft0= None, out_ft1=None, out_ft2=None, out_ft3=out_ft3)

            l2_v = myloss(v_out_end.view(batch_size, -1), y[:,:,1].view(batch_size, -1))
            l2_v.backward() # use the l2 re

            optimizer2.step()
            train_l2_v += l2_v.item()

# # Test:
    model1.eval()
    model2.eval()

    scheduler1.step()
    scheduler2.step()

    test_l2_u = 0.0
    i = 0
    for x, y in test_loader:
        x, y = x.cuda(), y.cuda()
        v_test_out_mid, out_ft0, out_ft1, out_ft2, out_ft3 = model2(x[:,:,1].unsqueeze(-1))
        u_test_out_end, out_ft0, out_ft1, out_ft2, out_ft3 = model1(x[:,:,0].unsqueeze(-1), out_ft0= None, out_ft1=None, out_ft2=None, out_ft3=out_ft3)
        u_test_out_end = u_test_out_end.view(batch_size, -1)
        y = y[:,:,0].view(batch_size, -1)
        if i == 0:
            u_pred =  u_test_out_end
            u_label = y
            i += 1
        else:
            u_pred = torch.cat((u_pred, u_test_out_end),0)
            u_label = torch.cat((u_label, y), 0)
        
        test_l2_u += myloss(u_test_out_end, y).item()

    train_l2_u /= ntrain
    test_l2_u /= ntest
    error_u.append(test_l2_u)

    test_l2_v = 0.0
    i = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()
            u_test_out_mid, out_ft0, out_ft1, out_ft2, out_ft3 = model1(x[:,:,0].unsqueeze(-1))
            v_test_out_end, out_ft0, out_ft1, out_ft2, out_ft3 = model2(x[:,:,1].unsqueeze(-1), out_ft0= None, out_ft1=None, out_ft2=None, out_ft3=out_ft3)
            v_test_out_end = v_test_out_end.view(batch_size, -1)
            y = y[:,:,1].view(batch_size, -1)
            if i == 0:
                v_pred =  v_test_out_end
                v_label = y
                i += 1
            else:
                v_pred = torch.cat((v_pred, v_test_out_end),0)
                v_label = torch.cat((v_label, y), 0)
            test_l2_v += myloss(v_test_out_end, y).item()

    train_l2_v /= ntrain
    test_l2_v /= ntest
    error_v.append(test_l2_v)

    print(ep,train_l2_u, train_l2_v,test_l2_u, test_l2_v)